# Faculty Load Report

SWU uses Jenzabar as our Student Information System (SIS) and each semester I use the SQL Server Reporting Services (SSRS) pre-created reports to pull data to calculate faculty load in preparation for payroll. The process of preparting load and payroll takes about a week's worth of work cleaning data from the SSRS reports, coordinating with faculty and chairs, and double/triple checking things with payroll. So, in the spirit of automation, I thought that I'd share this Python load report creator since it has greatly helped me reduce the time spent in preparting loads while also minimizing data entry and manipulation errors due to manually adjusting Excel spreadsheets (the old way!).

Enjoy!

rJb


In [ ]:
import pandas as pd

# insert a new csv file name as needed each time the SSRS report is created with fresh course information
courses = pd.read_csv('Fall_as_of_8-18-20.csv')

# courses # uncomment this line to display the data set

print('There are', courses.shape[0], 'rows by', courses.shape[1], 'columns in the unfiltered data set.')
total_len = courses.shape[0]

# Cancelled Courses

Display courses that have been cancelled and then remove from the data set.

In [ ]:
cancelled_courses = courses[courses['Cancelled'] == 'Cancelled'].copy()

# print(cancelled_courses['Title'].unique()) # uncomment to display list of cancelled courses

print('\n', len(cancelled_courses), 'cancelled courses')

In [ ]:
# remove cancelled courses from data set
courses = courses[courses['Cancelled'] != 'Cancelled']

print('There are now', courses.shape[0], 'courses listed after the canceled courses are filtered out.')

if total_len - len(cancelled_courses) == courses.shape[0]:
    print('\n Verified: the data set was filtered correctly.')
else:
    print('\n Error: the data set was not filtered properly.')

courses # uncomment to print the dataset

# Locate Specific School or College Courses

There are three distinct Schools/Colleges at SWU, each with supervision over three broad academic areas ranging from business, education, to arts and sciences. Within each area there are numerous subject areas taught, each indicated with a four-letter subject prefix such as "ENGL" (English), or "ACCT" (Accounting), or "EDUC" (Education).

## Subjects

Since the 'Course' column contains not only subject prefixes, but also four-digit course numbers and two-letter plus two-digit teaching modality and section number information, we will copy subject prefix information from the 'Course' column and create a new column called 'Subject'.

In [ ]:
# create a list, 'subjects', of all course prefixes
subjects = courses['Course'].str.split(' ').str.get(0)

print(subjects.unique()) # uncomment to display all subject prefixes

# add the 'subjects' list to the 'courses' data set as a new column called 'Subject' at position 3
courses.insert(3, 'Subject', subjects)

courses

Now we have a complete set of all courses with a new subject column next to the 'Course' column.

## Modality

We are almost all set to begin analysis, but notice that the 'Location' column only contains two codes: 'MC' (main campus) and 'ON' (online). Now that SWU has expanded our modalities to include additional hybrid 'HY', online traditional 'OT' options, we need to create a new column that clearly labels the modality. Modality information already exists for each course within the last two characters of the 'Course' column, so we'll start by extracting that information.

In [ ]:
# create a list of each course's modality
modalities = courses['Course'].str.split(' ').str.get(2)

# insert a new column, 'Modality' in position 5
courses.insert(5, 'Modality', modalities)

# show the unique values of modality
print(modalities.unique())

courses

As can be seen in the list above, many modalities match the pattern of two letters to indicate: hybrid 'HY', online 'ON', face-to-face 'FF', and online traditional 'OT'--followed by two numbers to indicate the section number of the course if there are multiple sections of the same course.

However, there are several other codes mixed into this list. 'OL' refers to the One Life program, but we'll go ahead and include this in the load report since SWU faculty sometimes teach in One Life as well and this should be considered when studying faculty loads.

Other codes warrant further consideration: any code starting single letter and ending with three numbers is an off-site dual enrollment course. These we'll consider along with all other codes except lunch codes added by Academic Records. These are coded under the Associate Registrar's name, so we can keep them in the data set and separate them according to her name.

# Faculty Load by School/College

The concept of facutly load applied to full-time faculty within the respective SWU academic units. So we'll calculate faculty load for each unit based on the directory of full-time faculty within each area.

- Note: later on, we will have to calculate payroll using a different strategy build on course subject prefixes because each academic unit "owns" certain subjects and must pay the faculty teaching them regardless of where the faculty are housed or if they are full-time or part-time.


In [ ]:
# create a list of course prefixes overseen by CAS
cas_prefixes = ['ASTH', 'BIBL', 'BIOL', 'CHEM', 'COMM', 'CPSC', 'CRJS', 
 'ENGL', 'EXSC', 'FRSC', 'HIST', 'MATH',  'ML', 'MUSC',  'PHED', 'PHSC', 'PHYS', 'PSYC', 'RECR', 'RELG', 'RSCH', 'SEMR',
 'SOSC', 'STAT']

# verify the length of the list of subject prefixes
print(len(cas_prefixes))

In [ ]:
# Pull out course, title, modality, enrollment, under a header for each faculty member
cols_needed = ['email_address','Title','Course','Course_Enrollment','Credit_Hours','Modality']

# create a new dataframe, "load", containing only the columns needed for faculty in the CAS faculty list
#alternate code# load = courses.loc[courses['email_address'].isin(cas_faculty), cols_needed]
load = courses.loc[courses['Subject'].isin(cas_prefixes), cols_needed]

# create a list of preferred sort categories and then sort
cols_sort = ['email_address','Course_Enrollment']
load = load.sort_values(cols_sort)

load.head(30)

From this point forward in the faculty load calculations, note that only college-specific subject prefixes are included in these calculations.

In [ ]:
# calculate the load for each course based on enrollment and course type; add a new 'load' column
load['Faculty_Load'] = load['Credit_Hours']

# create a Boolean mask to ID directed studies based on their unique Modality code
dir_study = load['Modality'].str.contains('000')

# set load to zero because DS courses are not part of load
load.loc[dir_study, 'Faculty_Load'] = 0 

# calculate pay for each directed study according to $150 per credit taken by each student
load.loc[dir_study, 'Overload_Pay'] = load.loc[dir_study, 'Course_Enrollment'] * load.loc[dir_study, 'Credit_Hours'] * 150

load.loc[dir_study]

As we can see above there are a handful of anticipated directed study courses (as in any semester). These represent courses that were originally planned as directed study courses. However, there are some courses each semester that we hope would fully populate with students (defined as five or more students), but do not meet sufficient enrollment and must be cancelled or counted as directed study courses. So, next, we'll identify courses that need to be converted into directed study courses and paid as such.

In [ ]:
# create a Boolean mask to ID courses to convert to directed studies based on their low enrollment
conv_dir_study = load['Course_Enrollment'] < 5

# set load to zero because DS courses are not part of load
load.loc[conv_dir_study, 'Modality'] = 'low_enroll' 

# display the courses that were flagged as low enrollment
load.loc[load['Modality'] == 'low_enroll']

## alternate code for forcibly adjusting loads
## set load to zero because DS courses are not part of load
#load.loc[conv_dir_study, 'Faculty_Load'] = 0

## display the courses that were newly adjusted to zero load
#load.loc[(load['Faculty_Load'] == 0) & (load['Course_Enrollment'] < 4) & (load['Overload_Pay'].isnull())]



We've made major adjustments to the 'load' dataset according to small enrollment courses, but we must consider that there are an assortment of applied lessons, internships/residencies, in addition to small courses.

- Applied lessons will be calculated as part of load because they are fee supported. They also must be calculated manually because this particular SSRS report does not pull the credit hour data per each student. Applied lessons students may enroll in 1 or 2 credit hours. So, we'll 1) calculate the faculty load as best we can, but then 2) mark applied lessons with a code 'approx' to flag them for later calculations.


In [ ]:
# Identify Applied Lessons and change 'Modality' label to "applied"
load.loc[(load['Title'].str.contains('Freshman')) & (load['Course'].str.contains('MUSC')), 'Modality'] = 'applied'
load.loc[(load['Title'].str.contains('Sophomore')) & (load['Course'].str.contains('MUSC')), 'Modality'] = 'applied'
load.loc[(load['Title'].str.contains('Junior')) & (load['Course'].str.contains('MUSC')), 'Modality'] = 'applied'
load.loc[(load['Title'].str.contains('Senior')) & (load['Course'].str.contains('MUSC')), 'Modality'] = 'applied'

# Identify and grant load for Diction Lab which is a special case mentioned in the faculty handbook (FHB)
load.loc[(load['Title'].str.contains('Diction')) & (load['Course'].str.contains('MUSC')), 'Faculty_Load'] = 1


# make a Boolean mask for applied lessons
applied_bool = load['Modality'] == 'applied'

# caluclate an approximate faculty load by multiplying enrollment by credit hours by 1/3 (studio load adjustment)
load.loc[applied_bool, 'Faculty_Load'] = load.loc[applied_bool, 'Course_Enrollment'] * load.loc[applied_bool, 'Credit_Hours'] * 1/3
load.loc[applied_bool, 'Comments'] = 'approx' 
load.loc[load['Comments'] == 'approx']

- Internships, residencies, and practica may have 1 to many students and must be calculated manually entirely because students can take such a broad range of credit hours. We'll mark these as 'manual' in the comments column.

In [ ]:
# Identify Internships, Practica, and Residencies and change 'Modality' label to "practicum"
load.loc[(load['Title'].str.contains('Internship')), 'Modality'] = 'practicum'
load.loc[(load['Title'].str.contains('Residency')), 'Modality'] = 'practicum'
load.loc[(load['Title'].str.contains('Practicum')), 'Modality'] = 'practicum'

# make a Boolean mask for applied lessons
practicum_bool = load['Modality'] == 'practicum'

print(load[practicum_bool].shape)
load[practicum_bool]

Now, we have also marked internship, practica, residency courses as "practicum" in the 'Modality' column--so we have the irregular load calculations of 'applied' and 'practicum' earmarked for further manual calculations as this load goes to payroll.

# Load Releases

Numerous faculty have load releases for a variety of reasons, including administrative duties or special projects. Next, we'll concatenate the load release information into the overall load report.

In [ ]:
# open a load release csv and read into a dataframe called 'releases'
releases = pd.read_csv('load_releases_cas_2021.csv')

# concatenate the load report and releases into a total load report
tot_load = pd.concat([load, releases], ignore_index=True)
tot_load

The load release information has been concatenated with the load data set. The 'email_address' columns and 'Faculty_Load' are in common and neatly line up in this new 'tot_load' data set.

In [ ]:
# use this optional export to Excel to test and validate the produced load report. Replace XXXX with the desired filename
tot_load.to_excel('load_report.xlsx')

First we'll create and overview of load per faculty member using a pivot table:

In [ ]:
import numpy as np
pv_load = tot_load.pivot_table(values=['Course_Enrollment', 'Faculty_Load'], index='email_address', aggfunc=[np.mean, np.sum], margins=True)
pv_load
pv_load.to_excel('load_summary.xlsx')

# create a list of CAS faculty using email addresses (less prone to formatting differences than full name)
cas_faculty = ['enter email addresses here as a list'] #enter emails here as a list

tot_load_ft = tot_load[tot_load['email_address'].isin(cas_faculty)]
pv_load_ft = tot_load_ft.pivot_table(values=['Course_Enrollment', 'Faculty_Load'], index='email_address', aggfunc=[np.mean, np.sum], margins=True)
pv_load_ft.to_excel('load_summary_ft.xlsx')


The two files above 'load_report' and 'load_summary' give us all we need to complete the final load calculations manually. Part of the manual process is to double check this automated process, but also the incoming data available from the SSRS SQL report is currently limited for certain courses such as applied lessons, internships, practica. So, we'll need to check those courses flagged in the 'load_report'.
